In [9]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
import tomli as tomllib
from transit import transit_validation_2019_alfaro_filepath
from transit_function import read_transit_assignments, read_dbf_and_groupby_sum
from bart import process_BART_data, process_BART_county, process_BART_SL
from muni import process_muni, read_transit_lines
from screen import concat_final_SL
from map_data import process_muni_map, process_bart_map
from obs import process_obs_data
from total_val import process_valTotal_Operator, process_valTotal_Submode, assign_ferry_name, assign_muni_name
from simwrapper_table import process_mkd_muni, process_mkd_bart, process_mkd_screenline, process_data

In [4]:

with open("transit.toml", "rb") as f:
    config = tomllib.load(f)

model_run_dir = Path(config["directories"]["model_run"])
markdown_output_dir = Path(config["directories"]["markdown_output_dir"])
transit_input_dir = Path(config["directories"]["transit_input_dir"])
observed_BART = Path(config["transit"]["observed_BART"])
observed_BART_county = Path(config["transit"]["observed_BART_county"])
observed_BART_SL = Path(config["transit"]["observed_BART_SL"])
observed_MUNI_Line = Path(config["transit"]["observed_MUNI_Line"])
observed_SL = Path(config["transit"]["observed_SL"])
observed_NTD = Path(config["transit"]["observed_NTD"])
model_BART = Path(config["output"]["model_BART"])
model_BART_county = Path(config["output"]["model_BART_county"])
model_BART_SL = Path(config["output"]["model_BART_SL"])
model_MUNI_Line = Path(config["output"]["model_MUNI_Line"]) 
model_SL = Path(config["output"]["model_SL"])
output_dir = model_run_dir / "validation_workbook" / "output"
output_transit_dir = output_dir / "transit"
transit_validation_2019_alfaro_filepath = config["transit"][
    "transit_validation_2019_alfaro_filepath"
]
transit_line_rename_filepath = (
    Path(config["directories"]["resources"]) / config["transit"]["line_rename_filename"]
)
time_periods = ["EA", "AM", "MD", "PM", "EV"]
tod_order = ['EA', 'AM', 'MD', 'PM', 'EV', 'Total']


In [13]:
combined_gdf = read_transit_assignments(model_run_dir, time_periods)

Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAEA.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAAM.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAMD.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAPM.DBF
Successfully read and added 'TOD' to: X:\Projects\CHAMP7\Run25z_Market\SFALLMSAEV.DBF


In [5]:
obs_MUNI_line = pd.read_excel(
    transit_validation_2019_alfaro_filepath,
    usecols="B:H",
    sheet_name="obs_MUNI_line",
    skiprows=list(range(9)),
)

In [11]:
line = read_transit_lines(model_run_dir, transit_line_rename_filepath)

In [12]:
line

,Name,Line
154,MUN108,108
155,MUN108AM,108
156,MUN25I,25
157,MUN25O,25
158,MUN108PM,108
...,...,...
469,MUNTISH2,TISH
470,MUNTISH3,TISH
471,MUNTO,T
472,MUNTSHTI,TSHT


In [15]:
muni = process_muni(combined_gdf, model_run_dir, transit_line_rename_filepath, transit_input_dir, observed_MUNI_Line, output_transit_dir, model_MUNI_Line)

In [16]:
muni